# Get Vacancy by Radius

### 12/29/17

- get vacantOSMNX_1516KenhillAve1000_122917

### 12/28/17

- delete Set Edge

- get vacantOSMNX_1516KenhillAve800_122817

- get vacantOSMNX_1516KenhillAve900_122917

In [1]:
# get clean vacant
import csv
import pandas as pd
import time
vacant_Baltimore = pd.read_csv('CleanDataBase/vacantBaltimore_171117_8000.csv',sep='\t') # create header later
vacant_Baltimore = vacant_Baltimore.drop("Unnamed: 0", 1)

# clean street name
vacant_Baltimore.loc[:,"street"] = vacant_Baltimore.loc[:,"street"].str[1:-1]
print "number of Vacancy in db25:", len(vacant_Baltimore)

number of Vacancy in db25: 8602


In [2]:
# get osmnx GEOdataframe
import osmnx as ox
address = '1516 Kenhill Ave, Baltimore, MD'
radius = 1000
gdf = ox.buildings_from_address(address, distance=radius)

print "Address:", address
print "radius", radius
print "number of buildings from osmnx:", len(gdf)

Address: 1516 Kenhill Ave, Baltimore, MD
radius 1000
number of buildings from osmnx: 7677


In [3]:
## check iff street is not null and building type is yes
result = pd.concat([gdf["addr:street"].isnull() == False, gdf["building"] == "yes" ], axis=1)
result["sum"] = result["addr:street"] & result["building"]
street_gdf = gdf[result["sum"]]

print "number of buildings with street name is not null:", len(street_gdf)

number of buildings with street name is not null: 7041


In [4]:
def vacant_generator(i,j):
    
    while i < len(street_gdf.index):
        while j < len(vacant_Baltimore.index):
            if (street_gdf['addr:street'][street_gdf.index[i]] == vacant_Baltimore['street'][vacant_Baltimore.index[j]] and 
                street_gdf['addr:housenumber'][street_gdf.index[i]] == vacant_Baltimore['housenumber'][vacant_Baltimore.index[j]]):
                
                yield street_gdf.index[i],vacant_Baltimore.index[j], street_gdf['addr:street'][street_gdf.index[i]], street_gdf['addr:housenumber'][street_gdf.index[i]]
            yield street_gdf.index[i],vacant_Baltimore.index[j],None,None
            j += 1
        
        if i%1000==0:
            print i
            
        i += 1
        j = 0
        
                #m.gdf.loc[i,'housetype'] = 2
            #elif m.gdf['housetype'][i] == 2:
                #m.gdf.loc[i,'housetype'] = 1

In [5]:
vacantOSMNX = []
vacantdb25 = []
s = time.time()
for i,j,street,housenumber in vacant_generator(0,0):
    if street != None:
        print street, housenumber, i, j
        vacantOSMNX.append(i)
        vacantdb25.append(j)
print "get", len(vacantOSMNX), "houses using", time.time() - s, "s"

0
Elmora Avenue 3632 336214473 6534
Elmley Avenue 3628 336215701 7133
Erdman Avenue 3902 336217363 1453
Erdman Avenue 3816 336219762 2264
Lyndale Avenue 3616 336219900 5898
Erdman Avenue 3624 336222155 3737
Lyndale Avenue 3563 336228666 8156
Lyndale Avenue 3545 336228674 5725
Erdman Avenue 3611 336228752 4623
Erdman Avenue 3607 336228755 4618
Elmora Avenue 3522 336232188 934
Elmley Avenue 3531 336232290 5014
Elmley Avenue 3526 336232736 3062
Elmley Avenue 3515 336233997 8231
Elmley Avenue 3506 336234316 2218
Edison Highway 2715 336235873 8286
Edison Highway 1218 336237580 7041
North Ellwood Avenue 1330 336241834 4551
East Federal Street 3024 336242110 3063
North Decker Avenue 1239 336243146 663
Elmley Avenue 3405 336243158 1635
Elmley Avenue 3402 336243292 2384
Elmley Avenue 3406 336243299 1345
North Decker Avenue 1210 336243636 2821
1000
Lyndale Avenue 3406 336243830 5547
Lyndale Avenue 3430 336243861 1264
Lyndale Avenue 3404 336243875 854
Ravenwood Avenue 3408 336244169 1992
Ravenwoo

East Biddle Street 2535 336262020 7500
East Biddle Street 2531 336262025 7497
North Luzerne Avenue 1306 336262043 1034
North Luzerne Avenue 1310 336262053 699
North Luzerne Avenue 1316 336262057 1153
East Chase Street 2512 336262107 4792
North Luzerne Avenue 1508 336262132 2166
North Luzerne Avenue 1326 336262249 5633
East Biddle Street 2514 336262265 2227
East Biddle Street 2530 336262267 2226
North Luzerne Avenue 1420 336262333 6
East Chase Street 2509 336262426 7756
East Madison Street 2504 336262493 7555
East Hoffman Street 2525 336262500 695
East Hoffman Street 2529 336262502 1699
East Hoffman Street 2523 336262503 1686
East Federal Street 2504 336262553 4102
North Rose Street 802 336262631 756
North Rose Street 814 336262634 610
East Eager Street 2518 336262680 7483
East Eager Street 2516 336262688 3995
East Hoffman Street 2516 336262706 7467
East Hoffman Street 2530 336262707 3389
East Hoffman Street 2526 336262709 3908
North Milton Avenue 739 336262819 2963
North Milton Avenue 

East Oliver Street 2331 336269751 3848
East Oliver Street 2335 336269756 2740
East Oliver Street 2309 336269783 3842
North Bradford Street 1719 336269846 5356
North Bradford Street 1717 336269853 2394
North Bradford Street 1727 336269869 1467
North Patterson Park Avenue 943 336270130 8214
North Patterson Park Avenue 945 336270135 3167
North Patterson Park Avenue 937 336270137 7135
North Patterson Park Avenue 933 336270138 8571
North Patterson Park Avenue 919 336270145 5927
North Patterson Park Avenue 903 336270148 4933
North Patterson Park Avenue 1103 336270431 7481
North Patterson Park Avenue 1105 336270439 7803
East Federal Street 2301 336270457 18
North Patterson Park Avenue 1203 336270582 3928
North Patterson Park Avenue 1211 336270584 3289
North Patterson Park Avenue 1201 336270592 63
North Patterson Park Avenue 1207 336270600 1096
North Patterson Park Avenue 834 336270737 1021
North Bradford Street 1610 336270805 6890
North Bradford Street 1622 336270806 2979
North Bradford Stree

In [6]:
print (time.time() - s)/60 , "mins"

42.3836955667 mins


In [7]:
len(vacantdb25)

563

In [8]:
# write data into csv
with open('VacantSet-OSMNX/vacantOSMNX_1516KenhillAve1000_122917', 'wb') as myfile:
    wr = csv.writer(myfile, quoting=csv.QUOTE_ALL)
    wr.writerow(vacantOSMNX)
    
"""
# Read data from csv
with open('vacantOSMNX_Baltimore_171117', 'rb') as f:
    reader = csv.reader(f)
    your_list = map(int,list(reader)[0])
""" ;

# Record